In [ ]:
# import libraries
!pip install tensorflow
# tensorflow_io 0.28 is compatible with TensorFlow 2.11
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 51.4 MB/s eta 0:00:00


In [ ]:
# import libaries
import io
import re
import string
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random
import os
import torch
#import pacakges
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
# Build the audio model
!pip install transformers==4.28.0
import tensorflow as tf
from transformers import TFAutoModel

# Load the pre-trained model using the Hugging Face interface
model_checkpoint = "facebook/wav2vec2-base"
huggingface_model = TFAutoModel.from_pretrained(model_checkpoint, trainable=False, from_pt=True)

# Define the inputs to the model
input_values = tf.keras.Input(shape=(16000,), dtype=tf.float32)

# Pass the inputs through the Wav2Vec model
wav2vec_output = huggingface_model(input_values)


# Create the TensorFlow functional API model
audio_model = tf.keras.Model(inputs=input_values, outputs=wav2vec_output)

# Print the model summary
audio_model.summary()

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(



TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tine this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_hid.weight', 'project_hid.bias', 'project_q.weight']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFWav2Vec2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFWav2Vec2Model were initialized from the PyTorch model.
If your task is similar to the task the model of the che

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 16000)]           0         
                                                                 
 tf_wav2_vec2_model (TFWav2V  TFWav2Vec2BaseModelOutpu  94371712 
 ec2Model)                   t(last_hidden_state=(Non            
                             e, 49, 768),                        
                              extract_features=(None,            
                              49, 512),                          
                              hidden_states=None, att            
                             entions=None)                       
                                                                 
Total params: 94,371,712
Trainable params: 0
Non-trainable params: 94,371,712
_________________________________________________________________


In [ ]:
from keras.models import Sequential

In [ ]:
# import word2vec vectors
from gensim.models import KeyedVectors
word2vec_vectors = KeyedVectors.load("/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/word2vec_embeddings/word2vec.wordvectors", mmap='r')

In [ ]:
aaaaimport pickle
# Save the vocabulary dictionary
with open('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented/vocab_dict.pkl', 'wb') as fp:
    pickle.dump(vocab, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [ ]:
# Import the vocabulary dictionary
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented/vocab_dict.pkl', 'rb') as handle:
    data = handle.read()
vocab = pickle.loads(data)

In [ ]:
vocab = tokenizer.word_index

In [ ]:
# Get the word embeddings for each word
#vocab = tokenizer.word_index
from gensim.models.keyedvectors import Word2VecKeyedVectors
import numpy as np
def get_weight_matrix():
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((len(vocab)+1, word2vec_vectors.vector_size))
    i=0
    for key in vocab.keys():
      if key=='OOV':
        continue
      elif key not in word2vec_vectors:
        i=i+1
        continue
      else:
        weight_matrix[i + 1] = word2vec_vectors[key]
        i=i+1
    return weight_matrix

In [ ]:
import numpy as np
embedding_vectors = get_weight_matrix()

In [ ]:
# Add an embedding layer to the text and time model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense
embedding_layer = Embedding(input_dim=len(vocab) + 1,
                                output_dim=300,
                                weights=[embedding_vectors],
                                input_length=50,
                                trainable=False)

In [ ]:
# Text and time model
# Define input layers
word_input = Input(shape=(50))
time_stamps = Input(shape=(50, 2))

# Define embedding layer

# Embed word and pos inputs
word_embedded = embedding_layer(word_input)


# Concatenate word and pos embeddings
concatenated = Concatenate()([word_embedded, time_stamps])

# Reshape the concatenated tensor
#reshaped = Reshape((-1, 16))(concatenated)

# Apply LSTM layer
lstm_output = LSTM(16, dropout=0.2, recurrent_dropout=0.2)(concatenated)

# Apply dense layer for binary classification
#output = Dense(1, activation='sigmoid')(lstm_output)

# Define the model
word_model = Model(inputs=[word_input, time_stamps], outputs=lstm_output, name='word_model')

# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
word_model.summary()

Model: "word_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      933000      ['input_4[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 50, 2)]      0           []                               
                                                                                                  
 concatenate_3 (Concatenate)    (None, 50, 302)      0           ['embedding[1][0]',              
                                                                  'input_5[0][0]']       

In [ ]:
# Concatenate audio, text and time models
from tensorflow.keras.layers import Reshape, RepeatVector, Concatenate
# Reshape word_model output to match the shape of audio_model output
audio_model_output = layers.GlobalAveragePooling1D()(audio_model.output[1])
# Drop-out layer before the final Classification-Head
audio_model_output = layers.Dropout(0.5) (audio_model_output)

concatenated_output = Concatenate()([audio_model_output, word_model.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
combined= Dense(1, activation='sigmoid')(concatenated_output)
combined_model = Model(inputs=[audio_model.input, word_model.input], outputs=combined, name='combined_model')
combined_model.summary()

Model: "combined_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 16000)]      0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 tf_wav2_vec2_model (TFWav2Vec2  TFWav2Vec2BaseModel  94371712   ['input_3[0][0]']                
 Model)                         Output(last_hidden_                                               
                                state=(None, 49, 76                                               
                                8),                                                  

In [ ]:
# Import datasets
from datasets import load_from_disk
final_combined_dataset=load_from_disk('/content/drive/MyDrive/Colab_Notebooks/dementia/English/dementia/English/Pitt/final_combined_data_original_augmented')
#final_combined_dataset=final_combined_dataset.train_test_split(test_size=0.2)
#final_trained_dataset=final_combined_dataset['train'].train_test_split(test_size=0.2)

In [ ]:
# Split the dataset into audio, text, and timestamps along with train, val, test sets
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Convert nested lists to numpy arrays
audio_train = pad_sequences(final_trained_dataset['train']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_train = np.asarray(audio_train).astype(np.float32)
audio_val = pad_sequences(final_trained_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_val = np.asarray(audio_val).astype(np.float32)
word_train = np.asarray(final_trained_dataset['train']['word']).astype(np.float32)
word_val = np.asarray(final_trained_dataset['test']['word']).astype(np.float32)
time_train = np.asarray(final_trained_dataset['train']['time_stamps']).astype(np.float32)
time_val = np.asarray(final_trained_dataset['test']['time_stamps']).astype(np.float32)
audio_test = pad_sequences(final_combined_dataset['test']['input_values'], maxlen=16000, dtype=np.float32, padding='post', truncating='post')
audio_test = np.asarray(audio_test).astype(np.float32)
word_test = np.asarray(final_combined_dataset['test']['word']).astype(np.float32)
time_test = np.asarray(final_combined_dataset['test']['time_stamps']).astype(np.float32)

In [ ]:
y_train=np.asarray(final_trained_dataset['train']['label']).astype(np.float32)
y_val=np.asarray(final_trained_dataset['test']['label']).astype(np.float32)
y_test=np.asarray(final_combined_dataset['test']['label']).astype(np.float32)

In [ ]:
combined_train = [word_train, time_train]
combined_val = [word_val, time_val]
combined_test = [word_test, time_test]

In [ ]:
# Compile and train the model
combined_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall(), tf.keras.metrics.AUC()])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=10,
                                            restore_best_weights=True)
combined_model.fit([audio_train, combined_train], y_train,
                       epochs=50, batch_size=16, validation_data=([audio_val, combined_val], y_val),
                       callbacks=callback)

Epoch 1/50
2041/2041 [==============================] - 391s 186ms/step - loss: 0.6819 - accuracy: 0.5665 - precision_1: 0.4971 - recall_1: 0.2391 - auc_1: 0.5571 - val_loss: 0.6696 - val_accuracy: 0.5863 - val_precision_1: 0.5641 - val_recall_1: 0.2275 - val_auc_1: 0.6024
Epoch 2/50
2041/2041 [==============================] - 372s 182ms/step - loss: 0.6425 - accuracy: 0.6100 - precision_1: 0.5657 - recall_1: 0.4209 - auc_1: 0.6503 - val_loss: 0.5921 - val_accuracy: 0.6576 - val_precision_1: 0.6165 - val_recall_1: 0.5694 - val_auc_1: 0.7308
Epoch 3/50
2041/2041 [==============================] - 370s 181ms/step - loss: 0.5834 - accuracy: 0.6658 - precision_1: 0.6101 - recall_1: 0.6289 - auc_1: 0.7360 - val_loss: 0.5756 - val_accuracy: 0.6700 - val_precision_1: 0.5905 - val_recall_1: 0.7947 - val_auc_1: 0.7524
Epoch 4/50
2041/2041 [==============================] - 368s 180ms/step - loss: 0.5563 - accuracy: 0.6962 - precision_1: 0.6422 - recall_1: 0.6710 - auc_1: 0.7705 - val_loss: 0.5

In [ ]:
# Evaluate the model using the test set
combined_model.evaluate([audio_test, combined_test], y_test)

319/319 [==============================] - 62s 188ms/step - loss: 0.3358 - accuracy: 0.8426 - precision_1: 0.8845 - recall_1: 0.7273 - auc_1: 0.9364


[0.33583396673202515,
 0.8425644636154175,
 0.8844654560089111,
 0.7273351550102234,
 0.9363961219787598]

In [ ]:
combined_model.save('/content/drive/MyDrive/Colab Notebooks/dementia/English/dementia/English/Pitt/trained_models_shorts_augmented/audio_word_time')